In [ ]:
import pandas as pd
import datetime
import random
from datetime import datetime, timedelta
from requests import get
import gtts
from playsound import playsound
import multiprocessing
import q

class Colection:
    def __init__(self):
        self.user = input('Inserta tu nombre ') #TODO pick from users
        self.words = pd.read_csv(directory+'\words.csv', sep = ';') #TODO words_user add
        self.suggestions = pd.read_csv(directory+'\words_suggestions.csv', sep = ';')[['spanish','english']]
        #self.words = pd.DataFrame()
        self.levels = {x : 2**x for x in range(9)}
        self.checks = {4 : {'er_mar' : 0.5, 'back' : 1}, 
                       5 : {'er_mar' : 0.3, 'back' : 1}, 
                       6 : {'er_mar' : 0.2, 'back' : 2}, 
                       7 : {'er_mar' : 0.1, 'back' : 2}, 
                       8 : {'er_mar' : 0.05, 'back' : 2}}

    def save_words(self):
        self.words.to_csv(directory+'\words.csv' ,sep= ';', index=False)        
        
    def scrape_meaning(self, word):
        while word[-1] == ' ':
            word = word[:-1]
        
        if len(word.split(' ')) > 1: #get rid of the article
            word = word.split(' ')[-1]
        
        word = word.replace(' ','')
        
        url = f'https://www.wordreference.com/definicion/{word}'
        
        my_headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OSX 10_14_3) AppleWebKit/537.36 (KHTML, like Gecko)Chrome/71.0.3578.98 Safari/537.36", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8"}
        html = str(get(url, headers = my_headers).text)
        html_split = html.split("</h3><ol class='entry'><li>")[1]

        def_list = []
        for x in html_split.split('</span><br>')[0].split('<li>'):
            if ':<br>' in x and '<span' not in x.split(':<br>')[0] and 'loc.' not in x:
                def_list.append(x.split(':<br>')[0])
                
        for num, definition in enumerate(def_list):
            print(f'definition {num}: {definition}')
            
        user_inp = input('Si quieres usar una de las definiciones, presione el numero de la definición')
        
        try:
            user_inp = int(user_inp)
        except:
            pass
        
        if type(user_inp) is int and -1 < user_inp < len(def_list):
            return def_list[user_inp]
        
        return None

    def add_word(self, word = None):      
        #Standard dict - this part will be the same for every record
        columns_none = ['last_review','error_margin','trials','level','errors'] #Q do we need a last review?
        record = {c : 0 for c in columns_none}
        record['next_review'] = datetime.now()
        
        #If a word was suggested with add_suggested_word function no user input is requiered
        if word:
            record['word'] = word
        else:
            record['word'] = input('Que palabra quieres agregar? ')
        
        #Try to get scrape definiton otherwise use user input
        scrape_description = self.scrape_meaning(record['word'])
        if scrape_description:
            record['description'] = scrape_description  
        else:
            record['description'] = input('Como describes la palabra? ')
        
        #Make audio file of description
        tts = gtts.gTTS(record['decription'], lang="es")
        tts.save(directory+fr"\audios\{record['word']}_description.mp3")
        
        #Make audio file of word
        tts = gtts.gTTS(record['word'], lang="es")
        tts.save(directory+fr"\audios\{record['word']}_word.mp3")        
        
        #Update the word list and save
        self.words = self.words.append(record, ignore_index=True)
        self.save_words()
        
    def add_suggested_words(self, amount = 10):
        d = self.suggestions.set_index('spanish').to_dict()['english']
        words = list(d.keys())
        random.shuffle(words)
        
        checked_words = []
        
        for word in words:
            if amount != 0:
                if input(f'Si quieres aprender {word}, (en ingles: {d[word]})?, presione clave "1"') == '1':
                    
                    self.add_word(word=word)
                    checked_words.append(word)
                    amount -= 1  
                else:
                    checked_words.append(word)
            else:
                break
                    
        self.suggestions = self.suggestions[~self.suggestions.spanish.isin(checked_words)]
        self.suggestions.to_csv(directory+'\words_suggestions.csv' ,sep= ';', index=False)
 
    def generic_test(self, d):
        errors = list(d.keys())
        error_list = []
        
        while len(errors) > 0: #Make seperate function of this part if the practice will be added
            new_errors = []
            random.shuffle(errors)
            for key in errors:
                p = multiprocessing.Process(target=playsound, args=(directory+fr"\audios\{key}_description.mp3",))
                p.start()
                print(d[key])
                is_error = False
                inp = input('¿Qué significa eso? ')
                p.terminate()
                p = multiprocessing.Process(target=playsound, args=(directory+fr"\audios\{key}_word.mp3",))
                p.start()
                if inp != key:
                    print (f'la respuesta correcta es: {key}')
                    new_errors.append(key)
                    is_error = True
                print('')
                error_list.append((key,datetime.now(),is_error))
                input('Pulse cualquier tecla para continuar')
                p.terminate()
            errors = new_errors 
            
        columns = ['word','date','is_error']
        df_practice_errors = pd.read_csv(directory+'\practice_errors.csv', sep = ';')[columns]
        df_errors = pd.DataFrame(error_list, columns= columns)
        df_practice_errors = df_practice_errors.append(df_errors)
        df_practice_errors.to_csv(directory+'\practice_errors.csv' ,sep= ';', index=False)        

    def practice_words(self, error_margin = 0.6, amount =10 , min_trials = 10):
        df =self.words[(self.words.error_margin > error_margin) & (self.words.trials >= min_trials)]
        df = df.sample(n=amount)
        d = df.set_index('word').to_dict()['description']
        self.generic_test(d)
        
    def practice_mistakes(self, days_back = False, error_margin = 0,amount = 1):
        df  = pd.read_csv(directory+'\practice_errors.csv', sep = ';')
        df.date = pd.to_datetime(df.date)
        
        if days_back:
            df = df[df.date > df.date - timedelta(days=days_back)]
        
        series = df.groupby('word')['is_error'].sum() / df.groupby('word').size()
        series = series > error_margin
        practice_set = list(series.index)
        practice_set = random.sample(practice_set,amount)
        
        test_words = self.words[self.words.word.isin(practice_set)]
        d = test_words.set_index('word').to_dict()['description']
        self.generic_test(d)
                      
    def test_words(self):
        def update_row(word, is_error):
            d = self.words[self.words.word == word].to_dict('records')[0]
            d['trials'] += 1
            if is_error is True:
                d['errors'] += 1

            d['error_margin'] = d['errors'] / d['trials']
            
            d['last_review'] = datetime.now()
            tdelta = self.levels[d['level']]
            d['next_review'] = datetime.now() + timedelta(days=tdelta)            
            
            if d['level'] in self.checks.keys() and d['error_margin'] >= self.checks[d['level']]['er_mar']:
                d['level'] = self.checks[d['level']]['back']
            elif d['level'] < 8:
                d['level'] +=1
                
            #Change the row
            self.words.loc[self.words['word'] == d['word']] = [pd.Series(d)]
        
        self.words.next_review = pd.to_datetime(self.words.next_review)
        test_words = self.words[self.words.next_review <= datetime.now()]
        d = test_words.set_index('word').to_dict()['description']
        errors = []
        preguntas = list(d.keys())
        random.shuffle(preguntas)
        
        for key in preguntas:
            p = multiprocessing.Process(target=playsound, args=(directory+fr"\audios\{key}_description.mp3",))
            p.start()
            print(d[key])
            inp = input('¿Qué significa eso? ') 
            p.terminate()
            
            p = multiprocessing.Process(target=playsound, args=(directory+fr"\audios\{key}_word.mp3",))

            if inp != key:
                print (f'la respuesta correcta es: {key}')
                p.start()
                print('')
                errors.append(key)
                update_row(key,True)
            else:
                p.start()
                update_row(key,False)
            input('Pulse cualquier tecla para continuar')
            p.terminate()
                
        self.save_words()
        
        d = {k:v for k, v in d.items() if k in errors}
        self.generic_test(d)
        
            
directory = r''
colection = Colection()

## The code below will let you practice your mistakes that you repeat most often
#### You can pick the amount of days that you want to go back

In [ ]:
colection.practice_mistakes(error_margin=0,days_back=2)

## The code below will let you practice your words with the highest error margin. 
#### Your mistakes will not count for the ladder. with min_trials you can say that you want to just run code that had at least a certain amount of trials on the ladder. With amount you can select the amount of words you want to practice and with error margin you can select a minimum error margin for words you want to practice.

In [ ]:
colection.practice_words(min_trials = 1, amount = 10)

## The code below let's you add any word you want to add

#### It will give you a suggestion for a definition or it allows you to add a definition yourself. While doing a test, the program will show the definition and you will have to enter the correct word

In [ ]:
colection.add_word()

## The code below let's you test the words

#### You will test the words on the 'ladder' which means that every time the spaced repitition gets bigger, except for the cases your error margin is to high to pass the checkpoint

In [ ]:
colection.test_words()

## The code below will let you add suggested words from the list of words that is already present in the directory

#### After it will give you an option for a definition that you can accept or you can pick to enter a definition yourself

In [ ]:
colection.add_suggested_words(amount = 1)